In [1]:
!pip install git+https://github.com/paobranco/ImbalanceMetrics.git

  Cloning https://github.com/paobranco/ImbalanceMetrics.git to c:\users\srtul\appdata\local\temp\pip-req-build-ah_wy1cu
  Resolved https://github.com/paobranco/ImbalanceMetrics.git to commit 190ed1d3a95fd55d37c51cabe2036b99416722e6
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/paobranco/ImbalanceMetrics.git 'C:\Users\srtul\AppData\Local\Temp\pip-req-build-ah_wy1cu'


# ImbalanceMetrics (Classification): Usage
## Example 1: Beginner

## Dependencies
First, we load the required dependencies. Here we import classification_metrics from imbalance_metrics to evalute the result we get from the SVM. In addition, we use pandas for data handling, and train_test_split to split the dataset.

In [2]:
# load dependencies
from imbalance_metrics import classification_metrics as cm
from sklearn import svm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

## Data
After, we load our data.


In [3]:
# load data
df = pd.read_csv(
    'https://raw.githubusercontent.com/paobranco/ImbalanceMetrics/main/data/glass.csv', header=None
)
df.head()

,0,1,2,3,4,5,6,7,8,9
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


In [4]:
# Assign x and y values from the dataframe
X = df.drop(columns=[9])
y = df[9]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

## Model
After, we train our model with data. In this example, we use the `svm.SVC()` from sklearn. This model will predict as y_pred which we will compare with true test value y in our evaluation.

In [6]:
clf = svm.SVC(kernel="linear", probability=True, random_state=0)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
y_proba=clf.predict_proba(X_test)

## Evaluation

Now we will evaluate the prediction using the functions from classification_metrics.

First, we are going calculate the relevance of each class by using `calculate_classification_phi` method. If we set return_phi_per_class as True, the function will only return relevance value of each class.

In [7]:
phi = cm.calculate_classification_phi(y_test, return_phi_per_class = True)
phi

2    0.045524
1    0.077390
7    0.103187
6    0.773900
Name: 9, dtype: float64

If we set return_phi as default False and phi_option as default 1, the function will return phi relevant weights by class.

In [8]:
y_phi = cm.calculate_classification_phi(y_test) # relevance by class frequency
y_phi

75     0.045524
84     0.045524
55     0.077390
120    0.045524
213    0.103187
         ...   
90     0.045524
15     0.077390
174    0.103187
30     0.077390
107    0.045524
Name: 9, Length: 71, dtype: float64

If we set phi_option as 2, the function will return alternative phi relevant weights by class where the phi relevance is divided by class frequency.

In [9]:
y_phi_alt = cm.calculate_classification_phi(y_test, phi_option = 2)
y_phi_alt

75     0.001339
84     0.001339
55     0.003869
120    0.001339
213    0.006879
         ...   
90     0.001339
15     0.003869
174    0.006879
30     0.003869
107    0.001339
Name: 9, Length: 71, dtype: float64

After we have calculated the Phi sample weights, we can pass them as sample_weight in `precision_score`, `recall_score` and `f1_score` etc.

In [10]:
precision_score(y_test, y_pred, average=None) # Without any weight

array([0.41860465, 0.68421053, 0.5       , 0.85714286])

In [11]:
precision_score(y_test, y_pred, average=None, sample_weight = y_phi) # With weight

array([0.49407966, 0.5104712 , 0.94444444, 0.44444444])

In [12]:
recall_score(y_test, y_pred, average=None) # Without any weight

array([0.9       , 0.38235294, 0.5       , 0.4       ])

In [13]:
recall_score(y_test, y_pred, average=None, sample_weight = y_phi) # With weight

array([0.9       , 0.38235294, 0.5       , 0.4       ])

In [14]:
f1_score(y_test, y_pred, average=None) # Without any weight

array([0.57142857, 0.49056604, 0.5       , 0.54545455])

In [15]:
f1_score(y_test, y_pred, average=None, sample_weight = y_phi) # With weight

array([0.63794302, 0.43721973, 0.65384615, 0.42105263])

Next, we are using `gmean_score` function to calculate the geometric mean score. First, we are setting weighted as default value True. That means the function will calculate the Phi relevance of each class and use them while calculating the geometric mean score.

In [16]:
cm.gmean_score(y_test, y_pred) # Weighted gmean 

0.0663348040214492

If we set weighted as False the function will only calculate geometric mean score without any weight.

In [17]:
cm.gmean_score(y_test, y_pred, weighted =False)

0.512193703259384

Furthermore, we are using Precision-Recall AUC using Davis method.

First, we will be using default value None as pos_label. When pos_label = None, the minority class is selected as pos_label. In this example, 1 is the minority class so here pos_label = 1.

In [18]:
# Here we set default None as positive label so the monority class can be seleted as positive class.
cm.pr_davis(y_test, y_proba)

0.7083333333333333

Next, we will be using value 7 as pos_label. This means 7 will be treated as the positive label for the dataset. This time we will also pass True as return_pr so the function also returns presicion and recall alongside davis auc.

In [19]:
# Here we set 0 as positive label
cm.pr_davis(y_test, y_proba,return_pr=True,pos_label=7)


(array([0.2238806 , 0.21212121, 0.21538462, 0.21875   , 0.22222222,
        0.22580645, 0.2295082 , 0.23333333, 0.23728814, 0.24137931,
        0.24561404, 0.25      , 0.25454545, 0.24074074, 0.24528302,
        0.25      , 0.23529412, 0.22      , 0.2244898 , 0.22916667,
        0.23404255, 0.23913043, 0.24444444, 0.25      , 0.25581395,
        0.26190476, 0.26829268, 0.275     , 0.28205128, 0.28947368,
        0.2972973 , 0.30555556, 0.31428571, 0.32352941, 0.33333333,
        0.34375   , 0.35483871, 0.36666667, 0.37931034, 0.35714286,
        0.33333333, 0.34615385, 0.36      , 0.33333333, 0.34782609,
        0.36363636, 0.33333333, 0.35      , 0.36842105, 0.38888889,
        0.41176471, 0.4375    , 0.46666667, 0.5       , 0.53846154,
        0.58333333, 0.63636364, 0.7       , 0.77777778, 0.75      ,
        0.85714286, 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        ]),
 array([1.        , 0.93333333, 0.93333333, 0.93333333, 0.93333333,
  

After Davis method, we are using Precision-Recall AUC using Manning method.

Like Davis method, we will be using default value None as pos_label first. This means minority class 6 will be treated as the positive label for the dataset.  

In [20]:
# Here we set default None as positive label so the monority class can be seleted as positive class.
cm.pr_manning(y_test, y_proba)

0.75

Next, we will be using value 7 as pos_label. This means 7 will be treated as the positive label for the dataset. This time we will also pass True as return_pr so the function also returns presicion and recall alongside manning auc.

In [21]:
# Here we set 0 as positive label
cm.pr_manning(y_test, y_proba,return_pr=True,pos_label=7)

(array([1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 0.85714286, 0.77777778, 0.77777778,
        0.7       , 0.63636364, 0.58333333, 0.53846154, 0.5       ,
        0.46666667, 0.4375    , 0.41176471, 0.38888889, 0.37931034,
        0.37931034, 0.37931034, 0.37931034, 0.37931034, 0.37931034,
        0.37931034, 0.37931034, 0.37931034, 0.37931034, 0.37931034,
        0.36666667, 0.35483871, 0.34375   , 0.33333333, 0.32352941,
        0.31428571, 0.30555556, 0.2972973 , 0.28947368, 0.28205128,
        0.275     , 0.26829268, 0.26190476, 0.25581395, 0.25454545,
        0.25454545, 0.25454545, 0.25454545, 0.25454545, 0.25454545,
        0.25454545, 0.25454545, 0.25454545, 0.25454545, 0.25454545,
        0.25454545, 0.25      , 0.24561404, 0.24137931, 0.23728814,
        0.23333333, 0.2295082 , 0.22580645, 0.2238806 , 0.2238806 ,
        0.2238806 , 0.2238806 , 0.2238806 , 0.22058824, 0.2173913 ,
        0.21428571, 0.21126761]),
 array([0.    

## Conclusion

In this package, we have presented a set of evaluation metrics specifically designed for imbalanced domains. Our package, "ImbalanceMetrics", provides a comprehensive set of evaluation metrics to assess the performance of machine learning models trained on imbalanced datasets.

Our package includes several evaluation metrics that address the challenges of imbalanced domains. These metrics can provide a more accurate assessment of model performance than traditional metrics, which can be misleading in imbalanced domains.

To learn more about our package, please refer to the documentation, which includes detailed descriptions of all the available metrics and their usage.